# Time aggregation

For a climatology, there are different ways to aggregate data in time. Common ways are:
* monthly climatology, aggregating all observations per month
* seasonal climatology
* yearly climatology
* decadal climatology

If the data coverage is sufficient, one can also make a seasonal climatology per decades which allows to resolve the seasonal cycle and long term changes.

In `DIVAnd`, the temporal aggregation is represented by a structure called time selector. The most common is `TimeSelectorYearListMonthList` which behaves similarly than the `yearlist` and `monthlist` files the Fortran version of DIVA.

In [ ]:
using Dates
using DIVAnd

In [ ]:
# ?TimeSelectorYearListMonthList

In [ ]:
yearlist = [1900:2017]
monthlist = [1:3,4:6,7:9,10:12]

TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist)

The number of time instances defined in this time selector is 4:

In [ ]:
length(TS)

Assume that we have a time vector with these dates:

In [ ]:
obstime = [DateTime(2001,4,1),DateTime(2002,2,1),DateTime(2018,3,1)]

Which observation would be used for the first winter analysis?

In [ ]:
sel = DIVAnd.select(TS,1,obstime)

In [ ]:
obstime[sel]

Note that 

A time instance in the "center" of a give time insterval is given by `DIVAnd.ctimes(TS)`. These dates are saved in the NetCDF file together with the `climatology_bounds` from the [NetCDF CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#climatological-statistics).

In [ ]:
DIVAnd.ctimes(TS)

In [ ]:
yearlist = [y:y+9 for y in 1950:10:2000]

Note that the duration of every year range is 10 years becasue the upper bound is inclusive. The last year range coveres the 10 years:

In [ ]:
collect(yearlist[end])'

In [ ]:
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);

For this time selector, there are now $4 × 6=24$ time slices

In [ ]:
length(TS)

In [ ]:
DIVAnd.ctimes(TS)[1:3]

# Overlapping years

Sometimes is it desirable to have overlapping year range to make a climatology similar to a running average. This can be achieved by a suitable definition of `yearlist`:

In [ ]:
yearlist = [y:y+5 for y in 1990:2000]

Every time slice is a 6-year average form data from the same season and there are $4 × 11=44$ time slices in this example. 

In [ ]:
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);
length(TS)

Since the data is overlapping, the same observation are used in multiple time instances:

In [ ]:
obstime = [DateTime(2000,1,1)]
for n = 1:length(TS)
    nobs = sum(DIVAnd.select(TS,n,obstime))
    if nobs > 0
        println("$nobs observation(s) are used in time slice $n")
    end
end

As expected an observations is used 6 times.